In [2]:
import os
import pandas as pd
from ete3 import Tree
from collections import Counter

os.getcwd()

'/Volumes/AHN/captive_ape_microbiome/scripts/gyrb/analyses'

In [2]:
#gyrb inputs
os.chdir('/Volumes/AHN/captive_ape_microbiome/results/gyrb/')
metadata_file = 'inputs/metadata_gyrb_amp_meta_passing_samples.txt'
tax_table_file = 'inputs/ASVs_taxonomy.txt'
asv_table_file = 'inputs/ASVs_filtered_counts.tsv'


In [5]:
#read in asv table, get sample names and number for each asv
def is_HR(sampleNames,sample_type_dict):
    """given a list of sample names uses sample type dictionary to determine how many sample types are present
    designates ASVs as host restricted = 1 sample type or mixed = multiple sample types.
    Captive sample types are not considered so some clades will have a 0 sample type length and they can fall within
    host-restricted clades or mixed clades or neither"""
    sampleTypes = [sample_type_dict[name] for name in sampleNames]
    sampleTypes = [x.replace('non_western_','').replace('western_','') for x in sampleTypes]
    neutral_sampleTypes = ['captive_gorilla','captive_bonobo','captive_chimp','captive_orangutan']
    HR_sampleTypes = list(set(sampleTypes) - set(neutral_sampleTypes))
    HR_sampleNum = len([x for x in sampleTypes if x not in neutral_sampleTypes])
    CP_sampleTypes = list(set(sampleTypes) & set(neutral_sampleTypes))
    CP_sampleNum = len([x for x in sampleTypes if x in neutral_sampleTypes])
    CP_pres = True if len(CP_sampleTypes) > 0 else False
    if len(HR_sampleTypes) == 0:
        HR_cat,HR_type='Unique_CP','Unique_CP'
    if len(HR_sampleTypes) == 1: #identifies host-restricted clades
        HR_cat,HR_type='HR','HR_'+HR_sampleTypes[0]  
    if len(HR_sampleTypes) > 1: 
        HR_cat = 'NHR'
        if len(HR_sampleTypes) == 2:
            if 'human' in HR_sampleTypes:
                HR_type = 'NHR_human_single_wild_ape'
            else:
                HR_type = 'NHR_2_wild_apes'
        if len(HR_sampleTypes) == 3:
            if 'human' in HR_sampleTypes:
                HR_type = 'NHR_human_2_wild_apes'
            else:
                HR_type = 'NHR_3_wild_apes'
        if len(HR_sampleTypes) == 4:
            HR_type = 'NHR_4_hominids'
        
    return(HR_sampleTypes,HR_sampleNum,HR_cat,HR_type,CP_pres,CP_sampleTypes,CP_sampleNum)


def asv_hr_table(asv_table_file,metadata_file,tax_table_file):
    asv_table = pd.read_csv(asv_table_file,sep='\t',index_col=0)
    sampleNames = asv_table.apply(lambda row: list(row.index[row>0]),axis=1)
    asv_df = sampleNames.reset_index()
    asv_df.columns = ['ASV','sampleNames']
    asv_df['sampleNum'] = asv_df['sampleNames'].apply(lambda names: len(names))
    
    #add host restriction info
    metadata = pd.read_csv(metadata_file,sep='\t',index_col=None)
    sample_type_dict = dict(zip(metadata['X.SampleID'], metadata['Description'])) 
    hr = asv_df['sampleNames'].apply(lambda x: pd.Series(is_HR(x,sample_type_dict),
                                                         index=['HR_sampleTypes','HR_sampleNum','HR_cat','HR_type',
                                                                'CP_pres','CP_sampleTypes','CP_sampleNum']))
    asv_hr_df = asv_df.merge(hr,left_index=True, right_index=True)
    
    #add taxonomic info
    tax_table = pd.read_csv(tax_table_file,sep='\t',index_col=None)
    tax_table = tax_table[['ASV','Phylum','Order','Family','Genus']]
    asv_full = asv_hr_df.merge(tax_table,on='ASV',how='left')
    
    return(asv_full)
    
def output_summary_table(asv_full,outfile):
     #create summary table
    all_asvs = pd.Series(asv_full['HR_type'].value_counts(),name='ALL')
    cp_asv = pd.Series(asv_full[asv_full['CP_pres']==True]['HR_type'].value_counts(),name='CP')
    num_Gen = len(asv_full['Genus'].value_counts())
    top_Gen = pd.DataFrame(
            asv_full['Genus'].value_counts().index[:10],
            columns=['Genus'])
    gen_asvs = pd.DataFrame(top_Gen['Genus'].apply(
            lambda Genus: 
            pd.Series(asv_full[asv_full['Genus']==Genus]['HR_type'].value_counts(),name=Genus)
            )).T    
    gen_asvs.columns = list(top_Gen['Genus'])  
    res = pd.concat([all_asvs,cp_asv,gen_asvs],axis=1).fillna(0).T
    order_col = sorted(list(res.columns))
    print(order_col)
    res = res[order_col]
    res.to_csv(outfile,sep='\t')
    return(res)


In [4]:
asv_gyrb = asv_hr_table(asv_table_file,metadata_file,tax_table_file)
output_summary_table(asv_gyrb,'analyses/codiv_moeller_ASVs/gyrb_hr_asv_table.txt')

['HR_human', 'HR_wild_bonobo', 'HR_wild_chimp', 'HR_wild_gorilla', 'NHR_2_wild_apes', 'NHR_human_2_wild_apes', 'NHR_human_single_wild_ape', 'Unique_CP']


,HR_human,HR_wild_bonobo,HR_wild_chimp,HR_wild_gorilla,NHR_2_wild_apes,NHR_human_2_wild_apes,NHR_human_single_wild_ape,Unique_CP
ALL,6183.0,353.0,207.0,258.0,14.0,5.0,34.0,94.0
CP,27.0,0.0,4.0,0.0,2.0,2.0,10.0,94.0
g__Prevotella,2450.0,203.0,114.0,46.0,1.0,0.0,2.0,55.0
g__Bacteroides,940.0,1.0,14.0,0.0,0.0,5.0,20.0,12.0
Unassigned,199.0,94.0,41.0,181.0,11.0,0.0,2.0,11.0
g__Alistipes,515.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0
g__RC9,363.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
g__Parabacteroides,247.0,38.0,3.0,1.0,1.0,0.0,3.0,3.0
g__Bacteroides_A,264.0,0.0,17.0,2.0,0.0,0.0,0.0,3.0
g__Porphyromonas,243.0,1.0,2.0,0.0,0.0,0.0,0.0,4.0


In [8]:
clades_df=pd.read_csv('analyses/codiv_moeller_ASVs/full_tree_clades_collapsed_table.txt',sep='\t')
clades_df=clades_df.rename(columns={"cladeTax": "Genus"})
output_summary_table(clades_df,'analyses/codiv_moeller_ASVs/gyrb_hr_clade_table.txt')
#clades_df_15perc =  clades_df.loc[(clades_df.wild_bonobo > 0.15) | 
#              (clades_df.wild_chimp > 0.15)  |
#              (clades_df.wild_gorilla > 0.15)|
#              (clades_df.western_human > 0.15)|
#              (clades_df.non_western_human > 0.15)] 
#output_summary_table(clades_df_15perc,'analyses/codiv_moeller_ASVs/gyrb_hr_clade_15perc_table.txt')

['CP', 'MX', 'human', 'wild_bonobo', 'wild_chimp', 'wild_gorilla']


,CP,MX,human,wild_bonobo,wild_chimp,wild_gorilla
ALL,6.0,51.0,249.0,14.0,20.0,19.0
CP,6.0,19.0,25.0,0.0,2.0,0.0
Bacteroidaceae_Bacteroides,4.0,25.0,75.0,0.0,1.0,0.0
Bacteroidaceae_Prevotella,0.0,5.0,75.0,8.0,9.0,4.0
Tannerellaceae_Parabacteroides,0.0,3.0,18.0,1.0,2.0,0.0
Rikenellaceae_Alistipes,0.0,5.0,17.0,0.0,0.0,0.0
Bacteroidaceae_Unassigned,0.0,1.0,13.0,3.0,1.0,3.0
Bacteroidaceae_Bacteroides_B,0.0,3.0,15.0,0.0,0.0,0.0
Tannerellaceae_Unassigned,0.0,2.0,3.0,0.0,4.0,1.0
WCHB1-69_Unassigned,0.0,3.0,0.0,0.0,0.0,7.0


In [58]:
def multi_site_sp(cp_desc):
    sites = list(set([x.split('_')[2] for x in cp_desc]))
    sitesNum = len(sites)
    sitesMulti = 'multi_site' if sitesNum > 1 else 'single_site'      
    species = list(set([x.split('_')[1] for x in cp_desc]))
    speciesNum = len(species)
    speciessMulti = 'multi_sp' if speciesNum > 1 else 'single_sp' 
    return(sitesMulti + '_' + speciessMulti)
multi_site_sp(['captive_chimp_HOUZ','captive_gorilla_HOUZ'])

def captive_apes_asv_summary(asv_full,metadata_file):
    asv_cp = asv_full[asv_full['CP_pres']==True]
    metadata = pd.read_csv(metadata_file,sep='\t',index_col=None)
    metadata['Description_site'] = metadata['Description']+'_' +metadata['site_code']
    sample_type_site_dict = dict(zip(metadata['X.SampleID'], metadata['Description_site']))

    description_df = asv_cp['sampleNames'].apply(lambda l: pd.Series(
        [sample_type_site_dict[name] for name in l]).value_counts())
    description_df = description_df.fillna(0)  
    capt_desc = list(set(metadata['Description_site'][metadata['captivity_status']=='captive']))
    description_df = description_df[capt_desc]
    description_df['CP_sp_loc'] = description_df.apply(lambda row: list(row.index[row>0]),axis=1)
    description_df['numEnclosure'] = description_df['CP_sp_loc'].apply(lambda x: len(x))
    description_df['multi_site_sp'] = description_df['CP_sp_loc'].apply(lambda x:  
                                                                        multi_site_sp(x))
    asv_cp_table = asv_cp.merge(description_df,left_index=True,right_index=True)
    return(asv_cp_table)

asv_cp_table = captive_apes_asv_summary(asv_gyrb,metadata_file)
asv_cp_table_summary = asv_cp_table.groupby(['numEnclosure','multi_site_sp']).size().reset_index(name="count")
print(asv_cp_table_summary)
asv_cp_table_summary.to_csv('analyses/codiv_moeller_ASVs/numEnclosures_table.txt',sep='\t',index=False)


NameError: name 'asv_gyrb' is not defined

In [59]:
asv_table = pd.read_csv(asv_table_file,sep='\t',index_col=0)
asv_table = asv_table.reset_index().rename(columns={"index": "ASV"})
tax_table = pd.read_csv(tax_table_file,sep='\t',index_col=None)
asv_table_gen = asv_table.merge(tax_table[['ASV','Phylum','Family','Genus']],on='ASV',how='left')
asv_table_gen = asv_table_gen[asv_table_gen['Genus']!='Unassigned']

def shared_ASVs_common_gen(ind1,ind2,asv_table_gen):
    pw = asv_table_gen[['Phylum','Family','Genus','ASV',ind1,ind2]][
        (asv_table_gen[ind1]>0)|(asv_table_gen[ind2]>0)
        ]
    #print(pw)
    common_gen = list(
                 set(pw[pw[ind1]>0]['Genus']) & 
                 set(pw[pw[ind2]>0]['Genus']))
    pw= pw[pw['Genus'].isin(common_gen)]
    ind1_ASVs = len(pw[(pw[ind1]>0)])
    ind2_ASVs = len(pw[(pw[ind2]>0)])
    common_ASVs = len(pw[(pw[ind1]>0)&(pw[ind2]>0)])
    res = pd.Series([ind1,ind2,common_gen,ind1_ASVs,ind2_ASVs,common_ASVs],
              index=['ind1','ind2','common_gen','ind1_ASVs','ind2_ASVs','common_ASVs'])
    return(res)

shared_ASVs_common_gen('cp.bon.COLZ.01.Bt','cp.bon.COLZ.02.Bt',asv_table_gen)

def prop_shared(row):
    if row['common_ASVs'] > 0:
        return(row['common_ASVs']/min(row['ind1_ASVs'],row['ind2_ASVs']))
    else:
        return(0)

KeyError: "['cp.bon.COLZ.01.Bt', 'cp.bon.COLZ.02.Bt'] not in index"

In [6]:
from itertools import combinations

def get_sp_site_comp(ind1,ind2):
    sp = ('same_spec' if ind1.split('_')[1] == ind2.split('_')[1] else 'diff_spec')
    st = ('same_site' if ind1.split('_')[2] == ind2.split('_')[2] else 'diff_site')
    return(sp+'_'+st)
get_sp_site_comp('captive_bonobo_COLZ','captive_chimp_COLZ')


def pw_metadata_capt_samples(metadata_file):
    metadata = pd.read_csv(metadata_file,sep='\t',index_col=None)
    metadata['Description_site'] = metadata['Description']+'_' +metadata['site_code']
    sample_type_site_dict = dict(zip(metadata['X.SampleID'], metadata['Description_site']))

    cp_samples = metadata['X.SampleID'][metadata['captivity_status']=='captive']
    pw_df = pd.DataFrame(combinations(cp_samples, 2),columns=['ind1','ind2'])
    pw_df['desc_site_ind1'] = pw_df['ind1'].apply(lambda x:sample_type_site_dict[x])
    pw_df['desc_site_ind2'] = pw_df['ind2'].apply(lambda x:sample_type_site_dict[x])
    pw_df['full_desc_comp'] = pw_df['desc_site_ind1'] + '_vs_' + pw_df['desc_site_ind2']
    pw_df['sp_site_comp'] = pw_df.apply(lambda row: 
                                        get_sp_site_comp(row['desc_site_ind1'],row['desc_site_ind2']),
                                        axis=1)
    return(pw_df)

def add_tax_to_asv_table(asv_table_file,tax_table_file):
    asv_table = pd.read_csv(asv_table_file,sep='\t',index_col=0)
    asv_table = asv_table.reset_index().rename(columns={"index": "ASV"})
    tax_table = pd.read_csv(tax_table_file,sep='\t',index_col=None)
    asv_table_tax = asv_table.merge(tax_table[['ASV','Phylum','Order','Family','Genus']],on='ASV',how='left')
    return(asv_table_tax)
    
asv_table_tax = add_tax_to_asv_table(asv_table_file,tax_table_file)
print(len(asv_table_tax))

                  ind1                 ind2          desc_site_ind1  \
0  cp.ora.ATLZ.114.16s  cp.ora.ATLZ.115.16s  captive_orangutan_ATLZ   
1  cp.ora.ATLZ.114.16s  cp.ora.ATLZ.116.16s  captive_orangutan_ATLZ   
2  cp.ora.ATLZ.114.16s  cp.ora.ATLZ.117.16s  captive_orangutan_ATLZ   
3  cp.ora.ATLZ.114.16s  cp.ora.ATLZ.118.16s  captive_orangutan_ATLZ   
4  cp.ora.ATLZ.114.16s  cp.ora.ATLZ.119.16s  captive_orangutan_ATLZ   

           desc_site_ind2                                    full_desc_comp  \
0  captive_orangutan_ATLZ  captive_orangutan_ATLZ_vs_captive_orangutan_ATLZ   
1  captive_orangutan_ATLZ  captive_orangutan_ATLZ_vs_captive_orangutan_ATLZ   
2  captive_orangutan_ATLZ  captive_orangutan_ATLZ_vs_captive_orangutan_ATLZ   
3  captive_orangutan_ATLZ  captive_orangutan_ATLZ_vs_captive_orangutan_ATLZ   
4  captive_orangutan_ATLZ  captive_orangutan_ATLZ_vs_captive_orangutan_ATLZ   

          sp_site_comp  
0  same_spec_same_site  
1  same_spec_same_site  
2  same_spec_same_site 

In [114]:
pw_df = pw_metadata_capt_samples(metadata_file)
asv_table_tax = add_tax_to_asv_table(asv_table_file,tax_table_file)

asv_table_tax = asv_table_tax[asv_table_tax['Genus']!='Unassigned']
shared_ASV_all_genera = pw_df.apply(lambda row: 
                    shared_ASVs_common_gen(row['ind1'],row['ind2'],asv_table_tax),axis=1)
pw_shared_ASV_all_genera = pw_df.merge(shared_ASV_all_genera,on=['ind1','ind2'])                       
pw_shared_ASV_all_genera['prop_shared_ASVs'] = pw_shared_ASV_all_genera.apply(
    lambda row: prop_shared(row),axis=1)
pw_shared_ASV_all_genera.to_csv(
    'analyses/codiv_moeller_ASVs/gyrb_prop_shared_ASVs_all_gen_table.txt',
    sep='\t',index=False)


In [115]:
asv_table_Prevotella = asv_table_gen[asv_table_gen['Genus']=='g__Prevotella']
shared_ASV_Prevotella = pw_df.apply(lambda row: 
                    shared_ASVs_common_gen(row['ind1'],row['ind2'],asv_table_Prevotella),axis=1)
pw_shared_ASV_Prevotella = pw_df.merge(shared_ASV_Prevotella,on=['ind1','ind2'])
pw_shared_ASV_Prevotella['prop_shared_ASVs'] = pw_shared_ASV_Prevotella.apply(
    lambda row: prop_shared(row),axis=1)
pw_shared_ASV_all_genera.to_csv('analyses/codiv_moeller_ASVs/gyrb_prop_shared_ASVs_Prevotella_table.txt',sep='\t',index=False)

# 16S

In [7]:
os.chdir('/Volumes/AHN/captive_ape_microbiome/results/16s/')
metadata_file = 'inputs/16S_metadata.txt'
tax_table_file = 'inputs/ASVs_taxonomy.txt'
asv_table_file = 'inputs/ASV_tab.txt'

In [8]:
asv_16s = asv_hr_table(asv_table_file,metadata_file,tax_table_file)
asv_16s.head()

output_summary_table(asv_16s,'analyses/tables/16s_hr_asv_table.txt')

['HR_human', 'HR_wild_bonobo', 'HR_wild_chimp', 'HR_wild_gorilla', 'NHR_2_wild_apes', 'NHR_3_wild_apes', 'NHR_4_hominids', 'NHR_human_2_wild_apes', 'NHR_human_single_wild_ape', 'Unique_CP']


,HR_human,HR_wild_bonobo,HR_wild_chimp,HR_wild_gorilla,NHR_2_wild_apes,NHR_3_wild_apes,NHR_4_hominids,NHR_human_2_wild_apes,NHR_human_single_wild_ape,Unique_CP
ALL,553.0,57.0,105.0,210.0,227.0,104.0,189.0,157.0,269.0,106.0
CP,227.0,0.0,10.0,8.0,20.0,19.0,166.0,128.0,187.0,106.0
Ruminococcaceae_UCG-014,18.0,0.0,1.0,1.0,2.0,3.0,19.0,11.0,14.0,3.0
Prevotella_9,38.0,2.0,7.0,2.0,1.0,1.0,0.0,0.0,9.0,0.0
Bacteroides,33.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,13.0,0.0
Rikenellaceae_RC9_gut_group,16.0,0.0,7.0,8.0,8.0,1.0,1.0,1.0,2.0,0.0
Treponema_2,14.0,0.0,4.0,6.0,1.0,1.0,0.0,0.0,3.0,12.0
Faecalibacterium,15.0,0.0,2.0,1.0,1.0,1.0,3.0,6.0,9.0,1.0
Ruminococcus_1,6.0,0.0,1.0,0.0,2.0,0.0,7.0,9.0,4.0,7.0
Ruminococcaceae_UCG-005,7.0,0.0,2.0,1.0,1.0,1.0,5.0,4.0,8.0,1.0


In [10]:
HR_Human_ASVs = asv_16s[asv_16s['HR_type']=='HR_human']
#how many captive ape species are they present in?
HR_Human_ASVs['CP_sampleTypes'].apply(lambda x: len(x)).value_counts()
#how many captive ape species are they present in?

HR_wild_ASVs = asv_16s[asv_16s['HR_type'].isin(['HR_wild_bonobo','HR_wild_chimp','HR_wild_gorilla'])]

HR_wild_ASVs_CP = HR_wild_ASVs[HR_wild_ASVs['CP_pres']==True]
HR_wild_ASVs_CP['HR_type'].value_counts()
HR_wild_ASVs_CP['CP_sampleTypes'].apply(lambda x: len(x)).value_counts()
HR_wild_ASVs_CP.to_csv('analyses/tables/16s_hr_in_cp_asv_table.txt',sep='\t')
#HR_wild_ASVs_CP['HR_type','CP_sampleTypes']


In [57]:
asv_16s = asv_hr_table(asv_table_file,metadata_file,tax_table_file)
output_summary_table(asv_16s,'analyses/tables/16s_hr_all_asvs_table.txt')

asv_Bifidobacteriales = asv_16s[asv_16s['Order']=='Bifidobacteriales']
output_summary_table(asv_Bifidobacteriales,'analyses/tables/16s_hr_Bifidobacteriales_asvs_table.txt')

asv_Bacteroidales = asv_16s[asv_16s['Order']=='Bacteroidales']
output_summary_table(asv_Bacteroidales,'analyses/tables/16s_hr_Bacteroidales_asvs_table.txt')

['HR_human', 'HR_wild_bonobo', 'HR_wild_chimp', 'HR_wild_gorilla', 'NHR_2_wild_apes', 'NHR_3_wild_apes', 'NHR_4_hominids', 'NHR_human_2_wild_apes', 'NHR_human_single_wild_ape', 'Unique_CP']
['HR_human', 'NHR_4_hominids', 'NHR_human_2_wild_apes', 'NHR_human_single_wild_ape']
['HR_human', 'HR_wild_bonobo', 'HR_wild_chimp', 'HR_wild_gorilla', 'NHR_2_wild_apes', 'NHR_3_wild_apes', 'NHR_4_hominids', 'NHR_human_2_wild_apes', 'NHR_human_single_wild_ape', 'Unique_CP']


,HR_human,HR_wild_bonobo,HR_wild_chimp,HR_wild_gorilla,NHR_2_wild_apes,NHR_3_wild_apes,NHR_4_hominids,NHR_human_2_wild_apes,NHR_human_single_wild_ape,Unique_CP
ALL,211.0,22.0,50.0,34.0,24.0,8.0,2.0,4.0,50.0,29.0
CP,83.0,0.0,0.0,2.0,3.0,0.0,1.0,3.0,28.0,29.0
Prevotella_9,38.0,2.0,7.0,2.0,1.0,1.0,0.0,0.0,9.0,0.0
Bacteroides,33.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,13.0,0.0
Rikenellaceae_RC9_gut_group,16.0,0.0,7.0,8.0,8.0,1.0,1.0,1.0,2.0,0.0
Prevotella_7,4.0,2.0,7.0,6.0,2.0,1.0,0.0,0.0,1.0,0.0
Alloprevotella,9.0,2.0,4.0,1.0,2.0,0.0,0.0,0.0,4.0,0.0
Prevotella_2,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Parabacteroides,6.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,0.0
Alistipes,12.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0


In [75]:
print(asv_cp_table[asv_cp_table['HR_cat']=='NHR']["multi_site_sp"].value_counts())
print(asv_cp_table[asv_cp_table['HR_cat']=='HR']["multi_site_sp"].value_counts())
print(asv_cp_table[asv_cp_table['HR_cat']=='Unique_CP']["multi_site_sp"].value_counts())

multi_site_multi_sp      398
single_site_single_sp     86
multi_site_single_sp      24
single_site_multi_sp      12
Name: multi_site_sp, dtype: int64
multi_site_multi_sp      165
single_site_single_sp     64
multi_site_single_sp      10
single_site_multi_sp       6
Name: multi_site_sp, dtype: int64
multi_site_multi_sp      70
single_site_single_sp    18
single_site_multi_sp     12
multi_site_single_sp      6
Name: multi_site_sp, dtype: int64


In [60]:
asv_cp_table = captive_apes_asv_summary(asv_16s,metadata_file)
asv_cp_table_summary = asv_cp_table.groupby(['numEnclosure','multi_site_sp']).size().reset_index(name="count")
print(asv_cp_table_summary)
asv_cp_table_summary.to_csv('analyses/tables/numEnclosures_table.txt',sep='\t',index=False)

cp_Bifidobacteriales = asv_cp_table[asv_cp_table['Order']=='Bifidobacteriales']
cp_Bifidobacteriales_summary = cp_Bifidobacteriales.groupby(['numEnclosure','multi_site_sp']).size().reset_index(name="count")
print(cp_Bifidobacteriales_summary)
cp_Bifidobacteriales_summary.to_csv('analyses/tables/numEnclosures_Bifidobacteriales_table.txt',sep='\t',index=False)

cp_Bacteroidales = asv_cp_table[asv_cp_table['Order']=='Bacteroidales']
cp_Bacteroidales_summary = cp_Bacteroidales.groupby(['numEnclosure','multi_site_sp']).size().reset_index(name="count")
print(cp_Bacteroidales_summary)
cp_Bacteroidales_summary.to_csv('analyses/tables/numEnclosures_Bacteroidales_table.txt',sep='\t',index=False) 

Prevotella_genera = ['Prevotella','Prevotella_2','Prevotella_7','Prevotella_1', 'Prevotella_9']
cp_Prevotella = asv_cp_table[asv_cp_table['Genus'].isin(Prevotella_genera)]
cp_Prevotella_summary = cp_Prevotella.groupby(['numEnclosure','multi_site_sp']).size().reset_index(name="count")
cp_Prevotella_summary.to_csv('analyses/tables/numEnclosures_Prevotella_table.txt',sep='\t',index=False) 


    numEnclosure          multi_site_sp  count
0              1  single_site_single_sp    168
1              2    multi_site_multi_sp     71
2              2   multi_site_single_sp     30
3              2   single_site_multi_sp     19
4              3    multi_site_multi_sp     92
5              3   multi_site_single_sp     10
6              3   single_site_multi_sp     11
7              4    multi_site_multi_sp     72
8              5    multi_site_multi_sp     72
9              6    multi_site_multi_sp     63
10             7    multi_site_multi_sp     56
11             8    multi_site_multi_sp     66
12             9    multi_site_multi_sp     66
13            10    multi_site_multi_sp     75
   numEnclosure        multi_site_sp  count
0             2  multi_site_multi_sp      2
    numEnclosure          multi_site_sp  count
0              1  single_site_single_sp     30
1              2    multi_site_multi_sp     11
2              2   multi_site_single_sp      5
3              2   

In [61]:
asv_cp_table.head()

,ASV,sampleNames,sampleNum,HR_sampleTypes,HR_sampleNum,HR_cat,HR_type,CP_pres,CP_sampleTypes,CP_sampleNum,...,captive_orangutan_ATLZ,captive_orangutan_HOUZ,captive_orangutan_COLZ,captive_gorilla_HOUZ,captive_bonobo_COLZ,captive_chimp_HOUZ,captive_gorilla_COLZ,CP_sp_loc,numEnclosure,multi_site_sp
5,ASV_224,"[cp.ora.ATLZ.114.16s, cp.ora.ATLZ.115.16s, cp....",293,"[wild_gorilla, wild_chimp, wild_bonobo, human]",238,NHR,NHR_4_hominids,True,"[captive_gorilla, captive_orangutan, captive_b...",55,...,6.0,2.0,0.0,7.0,5.0,8.0,8.0,"[captive_orangutan_COMZ, captive_chimp_PC, cap...",8,multi_site_multi_sp
6,ASV_840,"[cp.ora.COMZ.MB31.16s, cp.ora.COMZ.MB7.16s, cp...",37,[human],34,HR,HR_human,True,[captive_orangutan],3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[captive_orangutan_COMZ],1,single_site_single_sp
7,ASV_1451,"[cp.ora.ATLZ.117.16s, cp.ora.ATLZ.118.16s, cp....",34,"[wild_gorilla, wild_chimp, human]",25,NHR,NHR_human_2_wild_apes,True,"[captive_gorilla, captive_orangutan, captive_c...",9,...,4.0,0.0,0.0,1.0,0.0,1.0,1.0,"[captive_chimp_PC, captive_orangutan_ATLZ, cap...",5,multi_site_multi_sp
8,ASV_546,"[cp.ora.ATLZ.114.16s, cp.ora.ATLZ.115.16s, cp....",102,"[wild_chimp, human]",48,NHR,NHR_human_single_wild_ape,True,"[captive_gorilla, captive_orangutan, captive_b...",54,...,8.0,6.0,0.0,5.0,6.0,6.0,4.0,"[captive_gorilla_COMZ, captive_orangutan_COMZ,...",9,multi_site_multi_sp
9,ASV_1846,"[cp.ora.ATLZ.116.16s, cp.ora.ATLZ.123.16s, wd....",23,"[wild_chimp, human]",20,NHR,NHR_human_single_wild_ape,True,[captive_orangutan],3,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,"[captive_orangutan_ATLZ, captive_orangutan_COLZ]",2,multi_site_single_sp


In [121]:
def output_proportion_shared_ASVs(pw_df,asv_table_tax):
    shared = pw_df.apply(lambda row: 
                    shared_ASVs_common_gen(row['ind1'],row['ind2'],asv_table_tax),axis=1)
    pw_shared = pw_df.merge(shared,on=['ind1','ind2'])                       #pw_shared_ASV_all_genera

    pw_shared['prop_shared_ASVs'] = pw_shared.apply(
        lambda row: prop_shared(row),axis=1)
    return(pw_shared)

In [122]:
pw_df = pw_metadata_capt_samples(metadata_file)
asv_table_tax = add_tax_to_asv_table(asv_table_file,tax_table_file)
asv_table_tax = asv_table_tax[~asv_table_tax['Genus'].isna()]


pw_shared_ASV_all_genera = output_proportion_shared_ASVs(pw_df,asv_table_tax)
pw_shared_ASV_all_genera.to_csv(
    'analyses/tables/16s_prop_shared_ASVs_all_gen_table.txt',
    sep='\t',index=False)

In [124]:
asv_table_Bacteroidales = asv_table_tax[asv_table_tax['Order']=='Bacteroidales']
pw_shared_ASV_Bacteroidales = output_proportion_shared_ASVs(pw_df,asv_table_Bacteroidales)
pw_shared_ASV_Bacteroidales.to_csv(
    'analyses/tables/16s_prop_shared_ASVs_Bacteroidales_table.txt',
    sep='\t',index=False)

In [126]:
print(set(asv_table_Bacteroidales['Genus']))
Prevotella_genera = ['Prevotella','Prevotella_2','Prevotella_7','Prevotella_1', 'Prevotella_9']
asv_table_Prevotella = asv_table_tax[asv_table_tax['Genus'].isin(Prevotella_genera)]
pw_shared_ASV_Prevotella = output_proportion_shared_ASVs(pw_df,asv_table_Prevotella)
pw_shared_ASV_Prevotella.to_csv(
    'analyses/tables/16s_prop_shared_ASVs_Prevotella_table.txt',
    sep='\t',index=False)



{'Prevotella', 'Alloprevotella', 'Prevotellaceae_UCG-003', 'Prevotella_2', 'dgA-11_gut_group', 'Bacteroides', 'Barnesiella', 'Butyricimonas', 'Prevotellaceae_UCG-001', 'Prevotella_7', 'Rikenellaceae_RC9_gut_group', 'Prevotellaceae_NK3B31_group', 'Prevotellaceae_UCG-004', 'Alistipes', 'Prevotellaceae_Ga6A1_group', 'Coprobacter', 'CAG-873', 'Parabacteroides', 'Prevotella_1', 'Prevotella_9', 'Paraprevotella', 'Odoribacter'}


In [212]:
def prop_shared_by_order(order):
    asv_table_order = asv_table_tax[asv_table_tax['Order']==order]
    asv_table_order = output_proportion_shared_ASVs(pw_df,asv_table_order)
    asv_table_order['common_gen'].apply(lambda x: len(x)) #remove comparisons where two ind don't share gen
    asv_table_order = asv_table_order[asv_table_order['common_gen'].apply(lambda x: len(x)) > 0]
    asv_table_order['order']=order
    return(asv_table_order)

Clostridales = prop_shared_by_order('Clostridiales')

In [200]:
print(asv_16s[asv_16s['CP_pres']==True]['Order'].value_counts()[:10].index)


Index(['Clostridiales', 'Bacteroidales', 'Mollicutes_RF39',
       'Erysipelotrichales', 'Spirochaetales', 'Gastranaerophilales',
       'Selenomonadales', 'Betaproteobacteriales', 'Coriobacteriales',
       'Lactobacillales'],
      dtype='object')


2926

In [205]:
Bacteroidales = prop_shared_by_order('Bacteroidales')
Mollicutes_RF39 = prop_shared_by_order('Mollicutes_RF39')
Erysipelotrichales = prop_shared_by_order('Erysipelotrichales')
Spirochaetales = prop_shared_by_order('Spirochaetales')
Gastranaerophilales = prop_shared_by_order('Gastranaerophilales')
Selenomonadales = prop_shared_by_order('Selenomonadales')
Betaproteobacteriales = prop_shared_by_order('Selenomonadales')
Coriobacteriales = prop_shared_by_order('Coriobacteriales')
Lactobacillales = prop_shared_by_order('Lactobacillales')


In [214]:
pw_shared_ASV_all_genera['order']='all'
pw_shared_ASV_top10_gen = pd.concat([pw_shared_ASV_all_genera,Clostridales,Bacteroidales,Mollicutes_RF39,Erysipelotrichales,Spirochaetales,
 Gastranaerophilales,Selenomonadales,Betaproteobacteriales,Coriobacteriales,Lactobacillales])
pw_shared_ASV_top10_gen.to_csv(
    'analyses/tables/16s_prop_shared_ASVs_top10_table.txt',
    sep='\t',index=False)

In [210]:
pw_shared_ASV_top10_gen.head()

,ind1,ind2,desc_site_ind1,desc_site_ind2,full_desc_comp,sp_site_comp,common_gen,ind1_ASVs,ind2_ASVs,common_ASVs,prop_shared_ASVs,order
0,cp.ora.ATLZ.114.16s,cp.ora.ATLZ.115.16s,captive_orangutan_ATLZ,captive_orangutan_ATLZ,captive_orangutan_ATLZ_vs_captive_orangutan_ATLZ,same_spec_same_site,"[Intestinimonas, Ruminococcaceae_UCG-005, Rumi...",136,118,91,0.771186,all
1,cp.ora.ATLZ.114.16s,cp.ora.ATLZ.116.16s,captive_orangutan_ATLZ,captive_orangutan_ATLZ,captive_orangutan_ATLZ_vs_captive_orangutan_ATLZ,same_spec_same_site,"[Intestinimonas, Ruminococcaceae_UCG-005, Rumi...",141,131,96,0.732824,all
2,cp.ora.ATLZ.114.16s,cp.ora.ATLZ.117.16s,captive_orangutan_ATLZ,captive_orangutan_ATLZ,captive_orangutan_ATLZ_vs_captive_orangutan_ATLZ,same_spec_same_site,"[Intestinimonas, Ruminococcaceae_UCG-005, Rumi...",138,114,96,0.842105,all
3,cp.ora.ATLZ.114.16s,cp.ora.ATLZ.118.16s,captive_orangutan_ATLZ,captive_orangutan_ATLZ,captive_orangutan_ATLZ_vs_captive_orangutan_ATLZ,same_spec_same_site,"[Intestinimonas, Catenibacterium, Ruminococcac...",124,100,85,0.850000,all
4,cp.ora.ATLZ.114.16s,cp.ora.ATLZ.119.16s,captive_orangutan_ATLZ,captive_orangutan_ATLZ,captive_orangutan_ATLZ_vs_captive_orangutan_ATLZ,same_spec_same_site,"[Intestinimonas, Ruminococcaceae_UCG-005, Rumi...",104,72,54,0.750000,all


In [191]:
df2

,gen,prop_shared_ASVs,num_comp
0,0 0 Candidat...,NaN,0
1,0 0 Candidat...,0.927483,1772
2,0 0 Candidat...,0.575687,1297
3,0 0 Candidat...,1.000000,401
4,0 0 Candidat...,0.491477,528
5,0 0 Candidat...,0.868993,1712
